# Minimum
> Minimal way of managing a **(Directed Acyclic Graph)DAG pipeline**

In [1]:
# default_exp minimum

In [77]:
# export
from forgebox.config import Config
from datetime import datetime
from uuid import uuid4
from inspect import getargspec
from copy import deepcopy

In [43]:
def now():return datetime.now().timestamp()

class Link:
    is_link_identifier = True
    data = Config()
    inputs = Config()
    def __init__(self,func):
        self.ensure_funcs()
        func_name = func.__name__
        self.func_name = func_name
        self.funcs_[func_name] = func
        self.func = self.funcs_[func_name]
        self.get_kwargs_default()
        
    def __repr__(self,):
        return f"[Link Function]:{self.func}"
    
    def ensure_funcs(self,):
        if hasattr(self,"funcs_")==False:
            self.__class__.funcs_ = Config()
            
    def ensure_uuid(self,uuid):
        if uuid == False:
            uuid = str(uuid4())
            self.data[uuid] = Config()
            self.inputs[uuid] = Config()
            return uuid
        else:
            return uuid
        
    def check_callback(self,v,uuid):
        if hasattr(v,"is_link_identifier"):
            return v.unit_call(uuid)
        else:
            return v
        
    def unit_call(self,uuid = False):
        uuid = self.ensure_uuid(uuid)
        def called(*args,**kwargs):
            # record inputs
            kwargs0 = deepcopy(self.kwargs_default)
            kwargs0.update(kwargs)
            
            self.inputs[uuid].update(Config({self.func_name:Config(args = args,kwargs = kwargs0)}))
            
            kwargs0 = dict((k,self.check_callback(v,uuid) for k,v in kwargs0.items()))
            
            # run function
            rt = self.func(*args,**kwargs)
            
            # record outputs
            self.data[uuid].update({self.func_name:rt})
            return rt
        return called
        
    def __call__(self,*args,**kwargs,):
        return self.unit_call()(*args,**kwargs)
    
    def get_kwargs_default(self):
        arg_spec = getfullargspec(self.func)
        
        kwargs_default = dict()
        if arg_spec.defaults != None:
            kwargs_default.update(dict(zip(arg_spec.args[::-1],arg_spec.defaults[::-1])))
        if arg_spec.kwonlydefaults != None:
            kwargs_default.update(dict(zip(arg_spec.kwonlyargs[::-1],arg_spec.kwonlydefaults[::-1])))
        self.kwargs_default = kwargs_default

In [44]:
@Link
def abc(a = 2,b=3):
    return a**2

In [45]:
abc(5)

25

In [46]:
abc.inputs

{'e39bc0b1-781c-4e16-9c55-e76b5dc8d15e': {'abc': {'args': (5,), 'kwargs': {}}}}

In [52]:
abc.data

{'e39bc0b1-781c-4e16-9c55-e76b5dc8d15e': {'abc': 25}}

In [81]:
abc.funcs_

{'abc': <function __main__.abc(a=2, b=3)>}

In [13]:
from inspect import getargspec,getfullargspec

In [53]:
def a():return 123
def b(a,b=1):return 123
def c(*args,a=1,b=2):return a
def d(a=1,b:{"type":int}=2,**kwargs, ):return a
def e(a=1,**kwargs):return a

In [54]:
for f in [a,b,c,d,e]:
    print(getfullargspec(f))

FullArgSpec(args=[], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})
FullArgSpec(args=['a', 'b'], varargs=None, varkw=None, defaults=(1,), kwonlyargs=[], kwonlydefaults=None, annotations={})
FullArgSpec(args=[], varargs='args', varkw=None, defaults=None, kwonlyargs=['a', 'b'], kwonlydefaults={'a': 1, 'b': 2}, annotations={})
FullArgSpec(args=['a', 'b'], varargs=None, varkw='kwargs', defaults=(1, 2), kwonlyargs=[], kwonlydefaults=None, annotations={'b': {'type': <class 'int'>}})
FullArgSpec(args=['a'], varargs=None, varkw='kwargs', defaults=(1,), kwonlyargs=[], kwonlydefaults=None, annotations={})


In [56]:
arg_spec = getfullargspec(b)

kwargs_default = dict()
if arg_spec.defaults != None:
    kwargs_default.update(dict(zip(arg_spec.args[::-1],arg_spec.defaults[::-1])))
if arg_spec.kwonlydefaults != None:
    kwargs_default.update(dict(zip(arg_spec.kwonlyargs[::-1],arg_spec.kwonlydefaults[::-1])))

In [76]:
kwargs_default

{'b': 1}